In [1]:
# Quarto Alpha-Beta Search Tree Implementation
# Author: Mihir Phatak & Netra Inamdar
# MATH 8650 - Final Project Code 

minPlayer = -1
maxPlayer = 1
empty = -1
maxDepth = 5

timingList = []

import time
import matplotlib as plt
def alphaBetaPrune(state):
        #Run alpha-beta search on Quarto state with cuttof depth of 5
        maxDepth = 3
        return TreeSearchAlphaBeta(state, maxDepth, -float("inf"), float("inf"), 1)[1]



def TreeSearchAlphaBeta(state, depth, alpha, beta, player):
    
        #Return best moves found and AlphaBeta Values
        
        bestMove = state
        # If depth cutoff reached, heurustic of score is returned
        if state.gameOver():
            
                if player == maxPlayer:
                                winFor = "MIN"
                else:
                                winFor = "MAX"
                print "Victory / Winning state for " + winFor + " found in " + str(maxDepth-depth) + " move(s): " + state.render()
                return [(depth+1)*100000*-player, state]
        elif state.DrawGame():
                
                print "Tie state found in " + str(maxDepth-depth) + " move(s): " + state.render()
                return [(depth+1)*1000*-player, state]
        elif depth == 0:
                # Static Utility Used to improve search time
                return [MakeEstimateOfCost(state, player), state]
        if player == maxPlayer:
                moves = GetPossibleMoves(state)
                nextMove = moves.next()
                while nextMove != None:
                        nextState = PlayMove(nextMove, state)
                        result = TreeSearchAlphaBeta(nextState, depth-1, alpha, beta, not player)
                        if result[0] > alpha:
                                # If command line option toggled, print new alpha value.
                                print "MAX: old alpha=" + str(alpha) + ", new alpha=" + str(result[0])
                                alpha = result[0]
                  
                                bestMove = nextState
                        # Prune Tree for optimization
                        if beta <= alpha:
                                print "Pruning max."
                                break
                                # Check next move.
                        nextMove = moves.next()
                return [alpha, bestMove]

        else:
                moves = GetPossibleMoves(state)
                nextMove = moves.next()
                while nextMove != None:
                        nextState = PlayMove(nextMove, state)
                        result = TreeSearchAlphaBeta(nextState, depth-1, alpha, beta, not player)
                        if result[0] < beta:
              
                                print "MIN: old beta=" + str(beta) + ", new beta=" + str(result[0])
                                beta = result[0]
                                # Save current best move.
                                bestMove = nextState
                        # Prune tree serach if majority nodes are unproductive
                        if beta <= alpha:
                                print "Pruning min."
                                break
                        # Check next move.
                        nextMove = moves.next()
                return [beta, bestMove]


def PlayMove(move, state):
        # Apply move to state and return resulting game state. """
        cell = move[0]
        piece = move[1]
        newState = state.MakeCopy()
        newState.placePieceInSearch(cell)
        newState.setPieceToPlay(piece)
        return newState


def GetPossibleMoves(state):
        # Return generator of all possible moves, which are [cell, piece] pairs. """
        # Reverse list of remaining pieces to improve search time
        # Best moves are more likely to select pieces that are opposite in qualities to current piece.
        unplacedPieces = state.getRemainingPieces()
        unplacedPieces.reverse()
        for cell in state.getUnoccupiedCells():
                if len(state.getRemainingPieces()) == 1:
                        yield [cell, None]
                else:
                        for piece in unplacedPieces:
                                if piece != state.getPieceToPlay():
                                        yield [cell, piece]
        yield None


def GetRowCount(board, opponentPiece):
        """ Return heuristic cost for rows of given board. """
        emptyCells = 0
        commonOnes = empty
        commonZeroes = 0
        cost = 0
        for row in board:
                for cell in row:
                        if cell == empty:
                                # Count number of empty cells in row.
                                emptyCells += 1
                        else:
                                commonOnes = commonOnes & cell
                                commonZeroes = commonZeroes | cell
                if opponentPiece != None:
                        commonOnes = commonOnes & opponentPiece
                        commonZeroes = commonZeroes | opponentPiece
                if commonOnes > 0 or commonZeroes != ((1<<len(board))-1):
                        if emptyCells == 1:
                                # Two pieces in row and attribute(s) in common with piece to be played.
                                cost += 200
                elif emptyCells == 2:
                        # One piece in row and attribute(s) in common with piece to be played.
                        cost -= 50
        emptyCells = 0
        return cost



def getDiagonalCount(board, opponentPiece):
        #Return heuristic cost for diagonal of given board. """
        emptyCells = 0
        commonOnes = empty
        commonZeroes = 0
        cost = 0
        for i in range(len(board)):
                cell = board[i][i]
                if cell == empty:
                        # Count number of empty cells in diagonal.
                        emptyCells += 1
        else:
                commonOnes = commonOnes & cell
                commonZeroes = commonZeroes | cell
        if opponentPiece != None:
                commonOnes = commonOnes & opponentPiece
                commonZeroes = commonZeroes | opponentPiece
        if commonOnes > 0 or commonZeroes != ((1<<len(board))-1):
                if emptyCells == 1:
                        # Two pieces in diagonal and attribute(s) in common with piece to be played.
                        cost += 200
                elif emptyCells == 2:
                        # One piece in diagonal and attribute(s) in common with piece to be played.
                        cost -= 50
        return cost

def MakeEstimateOfCost(state, player):
        """ Return heuristic cost for current state. """
        board = state.board
        transposed = zip(*board)
        rev = list(board)
        rev.reverse()
        opponentPiece = state.getPieceToPlay()
        # Evaluate board based on the current placed pieces and piece to be given to opponent.
        cost = GetRowCount(board, opponentPiece) + GetRowCount(transposed, opponentPiece) + getDiagonalCount(board, opponentPiece) + getDiagonalCount(rev, opponentPiece)
        # A winning state that occurs in the fewest moves is preferred.
        return -player*cost


In [2]:
#Quarto State Class
# author: Mihir Phatak, Netra Inamdar

# State class is defined as a rows x rows board with attributes.
# For now rows and attributes are as specified as constants

rows = 3
attributes = 3

empty = -1
interCellGap = " "

class GameState:
    
#State represents the board, next piece to be played

    def __init__(self):
        # Making a new board
        global rows, attributes
        rows = 4
        attributes = 4
        self.rows = rows
        self.attributes = attributes
        self.pieces = pow(2, attributes)
        self.cells = self.rows * self.rows
        self.emptyRow = tuple([empty for col in range(rows)])
        self.board = tuple([self.emptyRow for row in range(rows)])

        # Initialization
        self.firstPiece = 0
        self.unplayedPieces = range(self.pieces)
        self.unoccupiedCells = range(self.cells)
        self.pieceToPlay = self.firstPiece

    def MakeCopy(self):
        # Make multiple Copies of The state
        newState = GameState()
        newState.unplayedPieces = list(self.unplayedPieces)
        newState.unoccupiedCells = list(self.unoccupiedCells)
        newState.pieceToPlay = self.pieceToPlay
        boardAsList = list(map(list, self.board))
        newState.board = tuple(map(tuple, boardAsList))
        return newState

    def setPieceToPlay(self, piece):
        # Set Piece On Board
        self.pieceToPlay = piece

    def getPieceToPlay(self):
        # Get Next Piece To Play
        return self.pieceToPlay

    def placePieceInSearch(self, cell):
        #Place piece in index cell during search
        # Prnts all possible states searched in console
        print "Searched state: placing " + str(self.pieceToPlay) + " in cell " + str(cell)
        self.place(cell)

    def placePieceInGame(self, cell):
        #Place the piece to play in indexed cell while playing
        print "Placing " + str(self.pieceToPlay) + " in cell " + str(cell)
        self.place(cell)

        
    def place(self, cell):
        # Place piece in indexed cell
        row = int(cell) // rows
        col = int(cell) % rows
        boardAsList = list(map(list, self.board))
        boardAsList[row][col] = self.pieceToPlay
        self.board = tuple(map(tuple, boardAsList))
        self.unplayedPieces.remove(self.pieceToPlay)
        self.unoccupiedCells.remove(cell)

    def getRemainingPieces(self):
        #Return the list of remaining pieces
        return self.unplayedPieces

    def formatPieces(self, pieces):
        # Display Pieces with Binary Representation
        formattedPieces = []
        for piece in pieces:
            formattedPieces.append(str(piece) + " (" + self.renderAsBitPatterns(piece) + ")")

        return formattedPieces

    def getUnoccupiedCells(self):
        # Get list of unoccupied cells
        return self.unoccupiedCells

    def calculateNextMove(self):
        #Get the best move
        return alphaBetaPrune(self)

    def isUnoccupied(self, cell):
        #Check whether cell is unoccupied
        return cell in self.unoccupiedCells

    def isUnplaced(self, piece):
        #Check if Piece has been played or not
        return piece in self.unplayedPieces

    def render(self):
        #Render state as string
        rendering = "Piece to play: " + str(self.pieceToPlay) \
            + " (" + self.renderAsBitPatterns(self.pieceToPlay) + ")\n"

        for row in range(0, rows):
            rowRendering = ""
            for col in range(0, rows):
                rowRendering += self.renderAsBitPatterns(self.board[row][col])
                rowRendering += interCellGap
            rendering += rowRendering + "\n"
        return rendering

    def renderAsBitPatterns(self, cell):
        # Show cell contents in binary format
        #print(cell)
        if cell == None:
            return "none"
        cellRendering = ""
        if cell == empty:
            for attribute in range(0, self.attributes):
                cellRendering = cellRendering + "_"
        else:
            for attribute in range(0, self.attributes):
                #print(cell)      
                cellRendering = ("0" if (cell) % 2 == 0 else "1") + cellRendering
                cell = (cell)
                cell /= 2


        #print(type(cellRendering))
        return cellRendering

    def WinningCheckRows(self, board):
        #Checks Winning Conditions
        for row in board:
            emptyCell = False
            commonOnes = empty
            commonZeroes = 0
            for cell in row:
                if cell == empty:
                    # If any cells are empty, the row can't be a win.
                    emptyCell = True
                    break
                else:
                    # If win, either commonOnes != 000 or commonZeros = bit string of all 1s
                    commonOnes = commonOnes & cell
                    commonZeroes = commonZeroes | cell
            if not emptyCell:
                if commonOnes > 0 or commonZeroes != ((1<<len(board))-1):
                    return True
        return False

    def WinningCheckColumns(self, board):
        #Check Columns for Win
        transposed = zip(*board)
        return self.WinningCheckRows(transposed)


    def WinningcheckLeftDiagonal(self, board):
        #Check Left Diagonal For Win
        emptyCell = False
        commonOnes = empty
        commonZeroes = 0
        for i in range(len(board)):
            cell = board[i][i]
            if cell == empty:
                # If any cells are empty, the diagonal can't be a win.
                emptyCell = True
                break
            # If win, either commonOnes != 0 or commonZeroes = bit string of all 1s
            commonOnes = commonOnes & cell
            commonZeroes = commonZeroes | cell
        if not emptyCell:
            return commonOnes > 0 or commonZeroes != ((1<<len(board))-1)

    def WinningcheckRightDiagonal(self, board):
        # Check right diagonal for win
        rev = list(board)
        rev.reverse()
        return self.WinningcheckLeftDiagonal(rev)

    def DrawGame(self):
        #Check if draw has occured
        return len(self.unplayedPieces) == 0

    def gameOver(self):
        #Check if Game was won
        boardAsList = list(map(list, self.board))
        return self.WinningCheckRows(boardAsList) or self.WinningCheckColumns(boardAsList) or self.WinningcheckLeftDiagonal(boardAsList) or self.WinningcheckRightDiagonal(boardAsList)


    def renderMappingBinaryToString(self,BinaryString):
        '''
        Quarto Piece Dictionary :  Size, HollowFlat, Color, Shape
        s - small
        T - tall
        H - Hollow
        F - Flat
        B - Black
        W - White
        R - Round
        S - square 
        '''
        MappingDictionary = { "0000" : 'sHBR',"1111":'TFWS',"0001" : "sHBS","1101":"TFBS", "0010":"sHWR", "0100": "sFBR", "1000":"THBR", "1100":"TFBR","0011":"sHWS","0111": "sFWS","1010":"THWR","0101":"sFBS","1001": "THBS","0110":"sFWR","1110" : "TFWR","1011": "THWS"}


        return(MappingDictionary[BinaryString])
    
    

In [3]:
## Quarto! CLI - Command Line Interface
# author: Netra Inamdar, Mihir Phatak
# purpose: AI Based Quarto Game Engine with Minimax AlphaBeta Pruning 

import sys

# Prints to terminal all the explored states and alpha/beta values
# This Quarto Algorithm relies heavily on utility functions for optimization, and uses depth search only upto 3 levels

userFirst = 1
computerFirst = 2

def main():
    """ Run the Quarto Playing Program! playing program. """
    print("Welcome to Quarto!")

    print "Let's Play!"
    print timingList
    playUntilExit()

def playUntilExit():
    """ Cotinue playing successive games until human player decides to stop. """
    while True:
        firstPlayer = getFirstPlayer()
        if firstPlayer == 0:
            print("Goodbye. Thanks for playing Quarto!.")
            return
        playQuarto(firstPlayer)

def getFirstPlayer():
    """ Loop over players during initialization """
    while True:
        response = raw_input("Which player goes first? (1 = you, 2 = computer, 0 = stop) ")
        if response == "1":
            return 1
        elif response == "2":
            return 2
        elif response == "0":
            return 0
        else:
            print("That is invalid Input. Please try input again.")

def playQuarto(firstPlayer):

    state = GameState()
    if firstPlayer == userFirst:
        userTurn(state)
    elif firstPlayer == computerFirst:
        computerTurn(state)
    else:
        assert "Avoid Recurrence"


def userTurn(state):
    """ Users Play simulation """
    print ("User's Turn!")
    print state.render()
    
    

    while True:

        cell = raw_input("Place piece " + str(state.getPieceToPlay()) + " at cell number: ")
        # Validate that input is an integer and corresponds to an unoccupied cell.
        # If not, prompt the user to enter another value.
        try:
            cell = int(cell)
            if state.isUnoccupied(cell):
                break
            else:
                print ("Invalid Entry. Please try again.")
        except ValueError:
            print ("Invalid Entry. Please try again.")

    state.placePieceInGame(cell)
    print state.render()

    # If game has been won, end game.
    if state.gameOver():
        print("User wins.")
        return
    elif state.DrawGame():
        print ("Game Tie!")
        return

    # If no pieces are left to play, return None.
    while True:

        listOfAvailablePieces = state.formatPieces(state.getRemainingPieces())
        for choice in listOfAvailablePieces:

            print(choice)
        nextPiece = raw_input("Choose next piece to play : \n ")
        # Validate that input is an integer and corresponds to an unplaced piece.
        # If not, prompt the user to enter another value.
        try:
            nextPiece = int(nextPiece)
            if state.isUnplaced(nextPiece):
                break
            else:
                print ("Invalid Entry. Please try again.")
        except ValueError:
            print messageTryAgain

    state.setPieceToPlay(nextPiece)
    computerTurn(state)
        

def computerTurn(state):
    # Computers play turn
    print ("Computer's turn.")
    
    print state.render()
    start = time.time() 
    # Use alpha-beta search to find best move.
    nextState = state.calculateNextMove()
    print nextState.render()

    # If game has not ended, continue gameplay.
    if nextState.gameOver():
        print "Computer Wins!"
        end = time.time()
        print("TIME")
        print(end - start)
        timingList.append(end-start)
        print(timingList)
        
        return
    elif nextState.DrawGame():
        print ("Game Tie!")
        print("TIME")
        end = time.time()
        print(end - start)
        timingList.append(end-start)
        #print (timingList)
        return
    else:
        print("TIME")
        end = time.time()
        print(end - start)
        timingList.append(end-start)
        #print (timingList)
        
        userTurn(nextState)
        

# Play game Main Function Called
main()
print timingList
       

Welcome to Quarto!
Let's Play!
[]


KeyboardInterrupt: 

In [ ]:
'''
Test Cases : 
In order to run evaluator through the game, we have refrained from completely automating the test cases. 

This is to ensure that the human play element in the game is preserved. 

We plan to present automated test cases during presentation. 

Enter following digits in the console prompt to test Quarto Play. 

Case 1 : 

1 0 1 2 2 4 4 6 3 15 5 9 10 - Computer Wins

Case 2: 
1 0 2 5 13 8 10 15 11 14 5 7 9 1 -  Computer Wins



'''